In [2]:
import TextExtractionScript
import llamaCall

# import mysqlCall

import mongoCall
import re
import json

In [32]:
text = TextExtractionScript.getText()
resultText = llamaCall.callLlama3(
    text
    # + "; suggest some complications that can happen in bullet points"
    + "; format the given text into a json format do not ignore any information; do not make up any information that isn't included in the document; do not use any special characters in the entire json; if you encounter blood pressure store it as separate key values(systolic and diastolic); if you encounter any + or - store it as a boolean value with key 'positive'; "
    # + """; extract vital paramters from the text, write me an sql query to insert data into mysql table named patient_data with schema
    #         +-----------------+-------------+------+-----+---------+-------+
    #         | Field           | Type        | Null | Key | Default | Extra |
    #         +-----------------+-------------+------+-----+---------+-------+
    #         | name            | varchar(50) | YES  |     | NULL    |       |
    #         | dob             | date        | YES  |     | NULL    |       |
    #         | account_number  | int         | YES  |     | NULL    |       |
    #         | temp            | float       | YES  |     | NULL    |       |
    #         | hr              | int         | YES  |     | NULL    |       |
    #         | bp              | float       | YES  |     | NULL    |       |
    #         | wt              | float       | YES  |     | NULL    |       |
    #         | ht              | float       | YES  |     | NULL    |       |
    #         | bmi             | float       | YES  |     | NULL    |       |
    #         | medical_history | text        | YES  |     | NULL    |       |
    #         +-----------------+-------------+------+-----+---------+-------+
    # """
)

In [34]:
resultText

'Here is the formatted JSON data:\n\n```\n{\n    "patient": {\n        "name": "Test, Hayley",\n        "dob": "1991-11-07",\n        "accountNumber": "17530"\n    },\n    "guarantor": {\n        "name": "Test, Hayley"\n    },\n    "pcp": {\n        "name": "Jennifer M Dyer"\n    },\n    "appointmentFacility": {\n        "name": "Dayton Internal Medicine Clinic Ine"\n    },\n    "reasonForAppointment": [\n        "Fever",\n        "Cough"\n    ],\n    "summaryView": {\n        "patient": "Hayley Test"\n    },\n    "historyOfPresentIllness": [\n        "Constitutional: Fatigue all day.",\n        "Night sweats new onset. Fever for 2-3 days with chills, low grade."\n    ],\n    "interimHistory": [\n        "Was hospitalized with a diagnosis of pneumonia."\n    ],\n    "vitalSigns": {\n        "temperature": 101.1,\n        "heartRate": 112,\n        "bloodPressure": {\n            "systolic": 143,\n            "diastolic": 82\n        },\n        "weight": 186,\n        "height": 175.26,

In [35]:
def extract_and_clean_text(text):
    # Regular expression to find text enclosed in triple backticks
    backtick_regex = r"```(.*?)```"
    code_blocks = re.findall(backtick_regex, text, re.DOTALL)

    code_blocks = re.sub(r'\n',r'',code_blocks[0])

    return code_blocks

In [36]:
finalQueryString = extract_and_clean_text(resultText)

In [38]:
finalQueryString

'{    "patient": {        "name": "Test, Hayley",        "dob": "1991-11-07",        "accountNumber": "17530"    },    "guarantor": {        "name": "Test, Hayley"    },    "pcp": {        "name": "Jennifer M Dyer"    },    "appointmentFacility": {        "name": "Dayton Internal Medicine Clinic Ine"    },    "reasonForAppointment": [        "Fever",        "Cough"    ],    "summaryView": {        "patient": "Hayley Test"    },    "historyOfPresentIllness": [        "Constitutional: Fatigue all day.",        "Night sweats new onset. Fever for 2-3 days with chills, low grade."    ],    "interimHistory": [        "Was hospitalized with a diagnosis of pneumonia."    ],    "vitalSigns": {        "temperature": 101.1,        "heartRate": 112,        "bloodPressure": {            "systolic": 143,            "diastolic": 82        },        "weight": 186,        "height": 175.26,        "bodyMassIndex": 27.46,        "respiratoryRate": 18,        "oxygenSaturation": 92,        "heightCm": 175

In [39]:
client = mongoCall.mongoCall()
#mongoCall.mongoInsertMany(client["mpc"], finalQueryString)

Pinged your deployment. You successfully connected to MongoDB!


In [31]:
finalQueryString[2420:2500]

' "peripheral_pulses": {"dorsalis_pedis": 2+},    "neurologic": ["nonfocal"],    '

In [42]:
finalQueryString=json.loads(finalQueryString)

In [43]:
mongoCall.mongoInsert(client["mpc"],finalQueryString)

InsertOneResult(ObjectId('66ce76d36b11a38ff987546a'), acknowledged=True)